In [1]:
# -*- coding: utf-8 -*-
import codecs
import re
import sys
import time
from collections import Counter, defaultdict, OrderedDict

# Preprocessing corpus

In [2]:
STOPWORDS_FILENAME = 'pap/stopwords1.txt'

stopwords = set([line.strip() for line in codecs.open(STOPWORDS_FILENAME, encoding='utf-8').readlines()])

In [3]:
def remove_special_chars(text):
    return re.compile(r'[\W\d]+', re.UNICODE).sub(' ', text)

In [4]:
def remove_whitespace(text):
    return re.compile(r'[\s\n]+', re.UNICODE).sub(' ', text)

In [5]:
def remove_stopwords(words):
    return [word for word in words if word not in stopwords]

In [6]:
def preprocess_corpus(corpus):
    corpus = corpus.strip().lower()
    corpus = remove_special_chars(corpus)
    corpus = remove_whitespace(corpus)
    words = corpus.split()
    return remove_stopwords(words)

In [7]:
text = u'Jesień w 2017 Japonii jest przepiękna.'
for word in preprocess_corpus(text):
    print word

jesień
japonii
przepiękna


In [8]:
FORMS_FILENAME = 'pap/odm.txt'

In [9]:
class Dictionary(object):
    def __init__(self):
        self.basic_forms = []
        self.forms = defaultdict(lambda: [])

        with codecs.open(FORMS_FILENAME, encoding='utf-8') as forms_file:
            lines = forms_file.readlines()
            for line in lines:
                forms = line.split(', ')
                id = len(self.basic_forms)
                basic_form = forms[0]
                self.basic_forms.append(basic_form)

                for form in forms:
                    self.forms[form].append(id)

    def rec(self, word):
        return self.forms[word]

    def bform(self, id):
        return self.basic_forms[id]

In [10]:
dict = Dictionary()

In [11]:
def basic_form(word):
    ids = dict.rec(word)
    return dict.bform(ids[0]) if len(ids) > 0 else word

In [12]:
print basic_form(u'drzewem')

drzewo


In [13]:
def get_basic_form(words):
    return [basic_form(word) for word in words]

In [14]:
text = [u'Jesień', u'Japonii', u'jest', u'przepiękna.']
for word in get_basic_form(text):
    print word

Jesień
Japonia
byæ
przepiękna.


In [15]:
CORPUS_PAP = 'pap/pap.txt'

In [16]:
with codecs.open(CORPUS_PAP, encoding='utf-8') as f:
    corpus = f.read().replace('\n', '')

In [62]:
# corpus

Getting the corpus with only relevant words.

In [17]:
start = time.time()
words = preprocess_corpus(corpus)
# words
words = get_basic_form(words)
stop = time.time()
print "Preprocess time: " + str(stop-start) + " s"

Preprocess time: 8.07610106468 s


In [18]:
denominator = len(words) #number of words in corpus
denominator

2022038

Count words frequency in corpus

In [19]:
word_frequency = Counter(words)
# word_frequency

In [20]:
word_frequency['rocznicy']

262

In [27]:
word_frequency['chleb']

49

In [28]:
word_frequency['jedzenie']

42

In [29]:
word_frequency['powszedni']

5

In [31]:
word_frequency[u'mas\u0142o']

1

Count the frequency of pair words

In [21]:
def get_pairs_parallel(word):
    offset = 12
    d =[]
    index = words.index(word) #get the index of the processed word
    if(index < len(words) - offset):
        for i in range(index, index+offset): #check the neighbours of the processed word in offset distance
            pair = (word, words[i+1]) #create a pair of co-occurance
            d.append(pair)
#                 print i, pair
    return d

In [22]:
from multiprocessing import Pool
start = time.time()
p = Pool(8)
d = p.map(get_pairs_parallel, words)
stop = time.time()
print "Getting pairs parallel time: " + str(stop-start) + " s"
print d[1:5]

Getting pairs parallel time: 14440.5019059 s
[[(u'\u015brody', u'czwartek'), (u'\u015brody', u'zmar\u0142'), (u'\u015brody', u'd\u0142ugiej'), (u'\u015brody', u'ci\u0119\u017ckiej'), (u'\u015brody', u'choroba'), (u'\u015brody', u'minister'), (u'\u015brody', u'kultura'), (u'\u015brody', u'dziedzictwo'), (u'\u015brody', u'narodowy'), (u'\u015brody', u'andrzej'), (u'\u015brody', u'zakrzewski'), (u'\u015brody', u'mia\u0142')], [(u'czwartek', u'zmar\u0142'), (u'czwartek', u'd\u0142ugiej'), (u'czwartek', u'ci\u0119\u017ckiej'), (u'czwartek', u'choroba'), (u'czwartek', u'minister'), (u'czwartek', u'kultura'), (u'czwartek', u'dziedzictwo'), (u'czwartek', u'narodowy'), (u'czwartek', u'andrzej'), (u'czwartek', u'zakrzewski'), (u'czwartek', u'mia\u0142'), (u'czwartek', u'wykszta\u0142cenia')], [(u'zmar\u0142', u'd\u0142ugiej'), (u'zmar\u0142', u'ci\u0119\u017ckiej'), (u'zmar\u0142', u'choroba'), (u'zmar\u0142', u'minister'), (u'zmar\u0142', u'kultura'), (u'zmar\u0142', u'dziedzictwo'), (u'zmar\u0

In [23]:
def count_pairs(lista):
    d = Counter()
    for pairs in lista:
        for pair in pairs:
            if(d[(pair[1], pair[0])] > 0): #check if there is already reversed pair
                pair = (pair[1], pair[0])
            d[pair] += 1
    return d       

In [24]:
start = time.time()
pairs_frequency = count_pairs(d)
stop = time.time()
print "Counting pairs time: " + str(stop-start) + " s"

Counting pairs time: 82.8877530098 s


In [ ]:
# pairs_frequency

In [25]:
pairs_frequency[('pomnik', 'premier')]

347

Sort the pairs according to their frequency in corpus

In [26]:
pairs_frequency.most_common(20)

[((u'czerwiec', u'polski'), 15441),
 ((u'polski', u'wynika\xe6'), 14583),
 ((u'polski', u'harmonogram'), 14583),
 ((u'polski', u'charkowie'), 14583),
 ((u'polski', u'wst\u0119pnego'), 14583),
 ((u'polski', u'obch\xf3d'), 14583),
 ((u'polski', u'katyniu'), 14583),
 ((u'polski', u'zbrodnia'), 14583),
 ((u'polski', u'miednoje'), 14583),
 ((u'polski', u'rocznicy'), 14583),
 ((u'polski', u'\u017co\u0142nierzy'), 14583),
 ((u'polski', u'lipiec'), 14583),
 ((u'proca', u'europejsk\u0105'), 11033),
 ((u'proca', u'uzgodni\xe6'), 11033),
 ((u'proca', u'strony'), 11033),
 ((u'proca', u'prognoza'), 11033),
 ((u'proca', u'oba'), 11033),
 ((u'proca', u'polski'), 11033),
 ((u'proca', u'podpisa\u0142y'), 11033),
 ((u'proca', u'rz\u0105d'), 11033)]

Find associations's frequency for given word

In [32]:
def find_associations(pairs, word):
    counter = Counter()
    for pair in pairs.most_common():
        if (word == pair[0][0]):
            counter[pair[0][1]] = pair[1]
        if (word == pair[0][1]):
            counter[pair[0][0]] = pair[1]
    return counter

In [34]:
find_associations(pairs_frequency, "premier").most_common(20)

[(u'europejski', 3649),
 (u'jerzego', 3502),
 (u'morze', 3502),
 (u'polski', 3502),
 (u'pier\u015bcienia', 3502),
 (u'pucku', 3502),
 (u'zako\u0144czy\u0142y', 3502),
 (u'srebrny', 3502),
 (u'rocznicy', 3502),
 (u'za\u015blubin', 3502),
 (u'buzka', 3502),
 (u'ba\u0142tyku', 3502),
 (u'obchody', 3502),
 (u'spotka\xe6', 2437),
 (u'w\u015br\xf3d', 1205),
 (u'wrze\u015bnia', 837),
 (u'nadal', 781),
 (u'wojny', 659),
 (u'wysoko\u015bci', 625),
 (u'\u015bwiatowej', 582)]

In [55]:
find_associations(pairs_frequency, "powszedni").most_common(20)

[(u'powa\u017cniejszych', 9),
 (u'taks\xf3wk\u0105', 5),
 (u'rad', 5),
 (u'ceny', 5),
 (u'warszawy', 5),
 (u'taks\xf3wkowy', 5),
 (u'nowa', 5),
 (u'miasto', 5),
 (u'zobowi\u0105za\u0142a', 5),
 (u'maksymalny', 5),
 (u'us\u0142ugi', 5),
 (u'warszawa', 5),
 (u'uchwali\u0142a', 5),
 (u'ortopeda', 3),
 (u'latniemal', 1),
 (u'miastawilna', 1),
 (u'tabliczk\u0105', 1),
 (u'je\u017ad\u017apomara\u0144czowym', 1),
 (u'ortopedyczny', 1),
 (u'wystawow\u0105', 1)]

In [56]:
find_associations(pairs_frequency, "jedzenie").most_common(20)

[(u'wywo\u0142ywa\u0142o', 42),
 (u'spo\u017cyciu', 42),
 (u'skutek', 42),
 (u'po\u017cywienia', 42),
 (u'tabletka', 42),
 (u'witamin\u0105', 42),
 (u'negatywny', 42),
 (u'witamin\u0119', 42),
 (u'zwi\u0119kszonej', 42),
 (u'dawka', 42),
 (u'bogaty', 42),
 (u'c', 42),
 (u'kurort', 31),
 (u'trzydziestu', 26),
 (u'znikn\u0119\u0142o', 6),
 (u'przeciws\u0142oneczne', 5),
 (u'policzek', 5),
 (u'nakt\xf3re', 5),
 (u'si\u0119bardziej', 5),
 (u'nvcjd', 4)]

In [57]:
find_associations(pairs_frequency, "chleb").most_common(20)

[(u'charga', 49),
 (u'pustynia', 49),
 (u'marcu', 49),
 (u'bada\u0142a', 49),
 (u'ofiar\u0119', 49),
 (u'zachodz\u0105ce', 49),
 (u'b\xf3stwo', 49),
 (u'zasypuj\u0105cej', 49),
 (u'przeznaczy\xe6', 49),
 (u'procesy', 49),
 (u'oaz\u0119', 49),
 (u'egipcie', 49),
 (u'ostrowie', 22),
 (u'najgorzej', 21),
 (u'nieszawie', 14),
 (u'kultowy', 12),
 (u'dob\xf3r', 11),
 (u'symbolizowa\xe6', 8),
 (u'gotowa\xe6', 8),
 (u'nada\u0142o', 8)]

In [58]:
find_associations(pairs_frequency, u'mas\u0142o').most_common(20)

[(u'smutniak', 1),
 (u'poziomieregionalnym', 1),
 (u'monitorowa\xe6', 1),
 (u'substancja', 1),
 (u'rozk\u0142adu', 1),
 (u'wykorzystywa\xe6', 1),
 (u'bartoszaobuchowicza', 1),
 (u'uczony', 1),
 (u'globalny', 1),
 (u'\u015brodowisko', 1),
 (u'zanieczyszczaj\u0105cych', 1),
 (u'zaorskiego', 1),
 (u'organiczny', 1),
 (u'izwi\u0105zk\xf3w', 1),
 (u'miazg\u0119', 1),
 (u'podaj\u0105', 1)]

Find the associantion's strenght for given word

In [35]:
def find_strenght(pairs, word, alpha = 0.66, beta = 0.00002, gamma = 0.00002):
    strenght_counter = Counter()
    for pair in pairs.most_common():
        if (word == pair[0][0]):
            association = pair[0][1]
        elif(word == pair[0][1]):
            association = pair[0][0]
        else:
            continue
        
        assoc_freq = word_frequency[association]
        pair_freq = pair[1]
        if (assoc_freq > beta * denominator):
            strenght = pair_freq / pow(assoc_freq, alpha)
        else:
            strenght = pair_freq / (gamma * denominator)
        
        strenght_counter[association] = strenght;
        
    return strenght_counter  

In [36]:
find_strenght(pairs_frequency, 'premier').most_common(10)

[(u'ba\u0142tyku', 218.32666053123876),
 (u'zako\u0144czy\u0142y', 119.96230549136857),
 (u'obchody', 117.63558292070766),
 (u'srebrny', 99.6132643823773),
 (u'rocznicy', 88.76408680854864),
 (u'pier\u015bcienia', 86.59580087021114),
 (u'pucku', 86.59580087021114),
 (u'za\u015blubin', 86.59580087021114),
 (u'morze', 76.99687930363577),
 (u'buzka', 67.69585201896305)]

In [37]:
for word in find_strenght(pairs_frequency, 'premier').most_common(10):
     print word[0],word[1]

bałtyku 218.326660531
zakończyły 119.962305491
obchody 117.635582921
srebrny 99.6132643824
rocznicy 88.7640868085
pierścienia 86.5958008702
pucku 86.5958008702
zaślubin 86.5958008702
morze 76.9968793036
buzka 67.695852019


In [38]:
for word in find_strenght(pairs_frequency, 'ekonomiczny').most_common(10):
     print word[0],word[1]

priorytet 23.7620351707
proca 23.6873147127
podpisały 20.8144298123
komisję 17.4781128062
gospodarczy 15.0530355364
uzgodniæ 14.7397757368
komitet 12.0841160891
wzrost 11.6571593886
nt 11.5075694932
tempo 10.3232947256


In [39]:
for word in find_strenght(pairs_frequency, 'polski').most_common(10):
     print word[0],word[1]

katyniu 1035.63686344
obchód 561.638806292
rocznicy 369.630690442
charkowie 360.601531722
wstępnego 360.601531722
miednoje 360.601531722
harmonogram 360.601531722
żołnierzy 262.130409974
zbrodnia 188.295597266
czerwiec 178.884953737


In [40]:
for word in find_strenght(pairs_frequency, 'kurs').most_common(10):
     print word[0],word[1]

zmienił 34.9316779337
spadł 27.0755477562
nfi 25.5039526423
wzrósł 25.4948014829
sesja 21.9747447183
rynku 17.1200590399
papier 16.7785652659
giełdy 16.3750331096
wartościowych 15.2843283467
funduszy 13.4052567965


In [41]:
for word in find_strenght(pairs_frequency, 'minister').most_common(10):
     print word[0],word[1]

badacz 210.573227332
prawnik 196.570510524
historyk 158.462304941
dziedzictwo 158.462304941
zakrzewski 102.223598172
wykształcenia 102.223598172
historia 58.7842587552
miał 47.6364995976
kultura 44.0475300737
andrzej 37.6390177757


In [42]:
for word in find_strenght(pairs_frequency, 'rosja').most_common(10):
     print word[0],word[1]

robertson 24.7161236137
zgadzaæ 24.0483403571
zakończenie 22.802670191
wizyty 18.9979880212
metoda 16.6858906453
stosowaæ 12.2216084023
czeczenii 9.26317676216
stolicy 8.30422702068
konflikt 7.48826151176
kwietniu 7.03944681833


In [43]:
for word in find_strenght(pairs_frequency, 'niemcy').most_common(10):
    print word[0],word[1]

przyjaciół 28.1124990628
służyć 23.0177943575
najważniejszych 16.7070774009
przykład 16.3323823029
cudowny 10.9295671001
przyjaźni 10.9295671001
mogącej 10.9295671001
izraela 8.28327431876
strefy 6.47300783992
obu 6.16915162033


In [44]:
for word in find_strenght(pairs_frequency, 'polityka').most_common(10):
    print word[0],word[1]

spowodować 7.96009516347
szansę 6.10498671093
braku 5.9641882783
uwolniæ 5.39240487032
świat 4.76607842058
wybuch 4.60887194086
wyrównywaæ 4.03058696226
groźby 4.03058696226
mogącej 4.03058696226
wspaniałą 4.03058696226


In [60]:
for word in find_strenght(pairs_frequency, u'mas\u0142o').most_common(15):
    print word[0],word[1]

środowisko 0.064257114166
globalny 0.0611448100415
substancja 0.0537001713157
wykorzystywaæ 0.0303918382779
uczony 0.0263523901014
smutniak 0.0247275273758
poziomieregionalnym 0.0247275273758
monitorowaæ 0.0247275273758
rozkładu 0.0247275273758
bartoszaobuchowicza 0.0247275273758
zanieczyszczających 0.0247275273758
zaorskiego 0.0247275273758
organiczny 0.0247275273758
izwiązków 0.0247275273758
miazgę 0.0247275273758


In [61]:
for word in find_strenght(pairs_frequency, 'jedzenie').most_common(15):
    print word[0],word[1]

bogaty 2.02362512064
c 1.30698462729
wywoływało 1.03855614979
spożyciu 1.03855614979
pożywienia 1.03855614979
tabletka 1.03855614979
witaminą 1.03855614979
witaminę 1.03855614979
zwiększonej 1.03855614979
dawka 1.03855614979
negatywny 0.952687731415
skutek 0.778436349142
kurort 0.766553348651
trzydziestu 0.642915711772
zniknęło 0.148365164255


In [62]:
for word in find_strenght(pairs_frequency, 'chleb').most_common(15):
    print word[0],word[1]

pustynia 4.03197019942
egipcie 3.97260898875
charga 1.21164884142
badała 1.21164884142
ofiarę 1.21164884142
zachodzące 1.21164884142
bóstwo 1.21164884142
zasypującej 1.21164884142
procesy 1.21164884142
oazę 1.21164884142
przeznaczyæ 1.02398450086
marcu 0.987209440264
ostrowie 0.544005602269
najgorzej 0.519278074893
nieszawie 0.346185383262


In [63]:
for word in find_strenght(pairs_frequency, 'powszedni').most_common(15):
    print word[0],word[1]

maksymalny 0.254684600063
poważniejszych 0.222547746383
usługi 0.196313297099
taksówką 0.123637636879
taksówkowy 0.123637636879
zobowiązała 0.123637636879
uchwaliła 0.123637636879
warszawy 0.085576047131
ceny 0.082078736656
ortopeda 0.0741825821275
miasto 0.0421890299733
rad 0.0282262695947
warszawa 0.0252665137081
nowa 0.024744106982
latniemal 0.0247275273758
